## Make requests with comma separated params

In [1]:
# Team 69
# Dillon Blake 1524907
# Andrea Delahaye 1424289
# Yue Peng 958289
# Jeff Phan 1577799
# Alistair Wilcox 212544

from unittest import TestCase
import requests
import json
import pickle
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

## Test the associated GET request

In [2]:
RUN_FROM = 'uni_wifi' #'bastion'
if RUN_FROM == 'bastion' : URL, HEADERS = 'http://fission:31001/', None
if RUN_FROM == 'uni_wifi': URL, HEADERS =  'http://172.26.135.52:9090/', {'HOST': 'fission'}

In [3]:
def get_model_from_api(model_name):
    response = requests.get(URL+f'models/{model_name}')
    return response

In [4]:
response = get_model_from_api('lin_reg_model_test')
response.text

'{"coef":[1.0,1.9999999999999993],"intercept":3.0000000000000018}\n'

In [5]:
def get_prediction_from_api(model_name, values):
    params = {'predictors': ','.join(map(str, values))}
    response = requests.get(URL+f'models/{model_name}', params=params)
    return response

In [6]:
# Prediction for a set of predictors
values = [1.1, 2.2]
response = get_prediction_from_api('lin_reg_model_test', values)
print(response.text)

{"prediction":8.5}



In [7]:
# Model name does not exist, should return BadParams
response = get_model_from_api('not_a_model_name')
print(response.text)

{"Status": 400, "Message": "There is no trained model stored with this name"}


In [8]:
# Too many predicors, should return BadParams
values = [1.1, 2.2, 3.3]
response = get_prediction_from_api('lin_reg_model_test', values)
print(response.text)

{"Status": 400, "Message": "Invalid Parameters"}


## Tests

In [13]:
URL, HEADERS =  'http://172.26.135.52:9090/', {'HOST': 'fission'}

PAGE_NOT_FOUND_STR = '404 page not found\n'
INVALID_REQUEST_STR = 'Invalid request'
BAD_PARAMS_STR = '{"Status": 400, "Message": "Invalid Parameters"}'
ERROR_STR = '{"Status": 500, "Message": "Internal Server Error"}'
EMPTY_STR = '{"Status": 200, "Data": []}'

In [10]:
class APITests(TestCase):

    def __init__(self) -> None:
        super().__init__()

    def test_models_api(self):
        # Missing header
        resp = requests.get(URL+'models').text
        self.assertEqual(resp,BAD_PARAMS_STR)

        # No predictors, returning model coefs
        resp = requests.get(URL+'models/test').json()
        self.assertAlmostEqual(resp['intercept'], 3.)
        self.assertAlmostEqual(resp['coef'][0], 1.)
        self.assertAlmostEqual(resp['coef'][1], 2.)

        # Too many predictors (here 3 instead of 2)
        params = {'predictors': '1.1,2.2,3.3'}
        resp = requests.get(URL+'models/test', params=params).text
        self.assertEqual(resp,BAD_PARAMS_STR)


        # Valid parameters
        params = {'predictors': '1.1,2.2'}
        resp = requests.get(URL+'models/test', params=params).json()
        self.assertAlmostEqual(resp['prediction'])

In [18]:
# Missing header
resp = requests.get(URL+'models').text
print((resp==PAGE_NOT_FOUND_STR))

# No predictors, returning model coefs
resp = requests.get(URL+'models/lin_reg_model_test').json()
print(resp)

# Too many predictors (here 3 instead of 2)
params = {'predictors': '1.1,2.2,3.3'}
resp = requests.get(URL+'models/lin_reg_model_test', params=params).text
print((resp==BAD_PARAMS_STR))


# Valid parameters
params = {'predictors': '1.1,2.2'}
resp = requests.get(URL+'models/lin_reg_model_test', params=params).json()
print((resp['prediction']))

True
{'coef': [1.0, 1.9999999999999993], 'intercept': 3.0000000000000018}
True
8.5


## Generate example model

In [12]:
# Generate some sample data
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
y = np.dot(X, np.array([1, 2])) + 3

# Train the linear regression model
model = LinearRegression().fit(X, y)

# Print the coefficients
print(f'Coefficients: {model.coef_}')
print(f'Intercept: {model.intercept_}')

Coefficients: [1. 2.]
Intercept: 3.0000000000000018


In [13]:
with open('lin_reg_model_test.pkl', 'wb') as file:
    pickle.dump(model, file)